In [5]:
from google.colab import drive
import zipfile


# Extract the dataset


zip_path = '/content/drive/MyDrive/Colab/Dataset.zip'
extract_path = '/content/dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [7]:
!pip install split-folders


Copying files: 0 files [00:00, ? files/s]


Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [14]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define source and destination paths
source_dir = "/content/dataset/Dataset"
train_dir = "/content/dataset/train"
test_dir = "/content/dataset/test"

# Create train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split each class folder
for class_folder in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_folder)
    if os.path.isdir(class_path):
        # List all images in class folder
        images = os.listdir(class_path)

        # Split into train and test (80% train, 20% test)
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Create class subdirectories in train and test
        os.makedirs(os.path.join(train_dir, class_folder), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_folder), exist_ok=True)

        # Copy files to train
        for img in train_images:
            shutil.copy(os.path.join(class_path, img),
                        os.path.join(train_dir, class_folder, img))

        # Copy files to test
        for img in test_images:
            shutil.copy(os.path.join(class_path, img),
                        os.path.join(test_dir, class_folder, img))

print("Dataset split completed.")


Dataset split completed.


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/test'

# Image size and batch size
img_size = 224
batch_size = 32

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255)

# Generating data from train directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'  # Since we have 3 classes
)

# Data augmentation for validation images
val_datagen = ImageDataGenerator(rescale=1./255)

# Generating data from validation directory
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'  # Since we have 3 classes
)

# Verify the class labels
print(train_generator.class_indices)


Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___healthy': 2}


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Model Architecture
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Second Convolutional Block
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Third Convolutional Block
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Fourth Convolutional Block
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Flattening Layer
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))

# Output Layer
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Model Summary
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 52, 52, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     2,359,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,499,203 (9.53 MB)

 Trainable params: 2,498,627 (9.53 MB)

 Non-trainable params: 576 (2.25 KB)

In [18]:
# Train the model
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator)

# Evaluate on validation data
loss, accuracy = model.evaluate(val_generator)

print("Loss:", loss)
print("Accuracy (Test Data):", accuracy * 100)


Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.9141 - loss: 0.5678 - val_accuracy: 0.3333 - val_loss: 2.3345
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.9171 - loss: 0.3408 - val_accuracy: 0.3333 - val_loss: 4.1913
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 83s 6s/step - accuracy: 0.9168 - loss: 0.4485 - val_accuracy: 0.3333 - val_loss: 4.4942
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.9499 - loss: 0.3595 - val_accuracy: 0.3333 - val_loss: 5.7789
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.9671 - loss: 0.1292 - val_accuracy: 0.3333 - val_loss: 7.8838
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.3264 - loss: 7.9266
Loss: 7.883790969848633
Accuracy (Test Data): 33.33333432674408


In [20]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Example image path for prediction (change as needed)
img_path = '/content/dataset/test/Tomato___Early_blight/0a2726e0-3358-4a46-b6dc-563a5a9f2bdf___RS_Erly.B 7860.JPG'

# Load and preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Rescale image

# Make prediction
prediction = model.predict(img_array)

# Class names as per the dataset structure
class_names = ['Tomato__bacterial_spot', 'Tomato__early_blight', 'Tomato__healthy']

# Get the predicted class
predicted_class = np.argmax(prediction)

print("Prediction:", prediction)
print("Predicted class:", class_names[predicted_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
Prediction: [[9.9979299e-01 2.0614103e-04 8.5873376e-07]]
Predicted class: Tomato__bacterial_spot
